원본: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SEQUENCE-MODELS-AND-LONG-SHORT-TERM-MEMORY-NETWORKS" data-toc-modified-id="SEQUENCE-MODELS-AND-LONG-SHORT-TERM-MEMORY-NETWORKS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS</a></span><ul class="toc-item"><li><span><a href="#LSTM-IN-PYTORCH" data-toc-modified-id="LSTM-IN-PYTORCH-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>LSTM IN PYTORCH</a></span></li><li><span><a href="#Example:-An-LSTM-for-Part-of-Speech-Tagging" data-toc-modified-id="Example:-An-LSTM-for-Part-of-Speech-Tagging-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Example: An LSTM for Part-of-Speech Tagging</a></span></li><li><span><a href="#Exercise:-Augmenting-the-LSTM-part-of-speech-tagger-with-character-level-features" data-toc-modified-id="Exercise:-Augmenting-the-LSTM-part-of-speech-tagger-with-character-level-features-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Exercise: Augmenting the LSTM part-of-speech tagger with character-level features</a></span></li></ul></li></ul></div>

 # SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS

지금까지 우리는 다양한 피드-포워드 네트워크를 봐왔습니다. 

이것은 우리가 원하지 않을지라도, 네트워크에 의해 유지되는 상태가 존재하지 않았습니다.

NLP의 핵심인 Sequence 모델은 시간과 Input 간에 의존적인 관계가 존재합니다.

그 예로 품사 태깅을 위한 Hidden Markov Model이나 Conditional Random Field가 있습니다.

Recurrent neural network은 이러한 상태 개념을 가지고 다니는 network 입니다. 

예를 들어 network의 output이 다음 input에 포함되어 진행되면서, 예전 정보가 연속적으로 network를 따라 계속 전파될 수 있는 것입니다.

LSTM의 경우, sequence의 각 ($t$번째) 요소마다 hidden state $h_t$를 갖게 되는데, 이 $h_t$는 원칙적으로 과거의 어떤 정보라도 가지고 있을 수 있다. 

우리는 이 $h_t$를 이용해서 language model을 구축해서 단어를 예측할 수도 있고, 품사 태깅을 할 수도 있게 된다.

## LSTM IN PYTORCH

본격적인 예제에 들어가기 전에, 몇 가지 알아둘 것이 있다.

Pytorch의 LSTM은 모든 input을 3D tensor라고 가정하는데, 이 세 가지 축이 의미하는 바를 알아두는 것이 중요하다.

첫 번쨰 축은 sequnece 그 자체이며,

두 번쨰 축은 mini-batch의 순서 정보를 의미하는데, 여기서는 mini-batch에 대해 다룬 적이 없으므로 무조건 1차원을 가진다고 가정하겠다.

세 번쨰 축은 Input의 Index를 갖는다.

만약 sequnece 모델에 'The cow jumped'라는 문장을 수행하고 싶다면 input은 아래와 같은 형태를 갖는다.


$$
\begin{bmatrix}
\overbrace{q_{\text{The}}}^{\text{row vector}} \\
q_{\text{cow}} \\
q_{\text{jumped}}
\end{bmatrix}
$$

1차원인 두 번쨰 축이 있음을 기억하자.

추가로, 한번에 sequence를 보내는 경우에는 첫 번쨰 축이 1차원이 될 것이다.

아래 간단한 예제를 살펴보자.

In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


## Example: An LSTM for Part-of-Speech Tagging

이번에는 LSTM을 이용해서 품사 태깅을 해보려고 한다. 

또한 여기서는 Viterbi나 Forward-Backward와 같은 알고리즘은 사용하지 않을 것이지만, LSTM이 어떻게 돌아가는 지 경험한 후에는 언급한 
알고리즘들이 어떻게 사용될 수 있을지 생각해보는 것도 독자들에게 좋은 (꽤 어려운) 연습 문제가 될 것 같다.

품사 태깅 모델을 설명하겠다.

Input 문장을 $w_1, \cdots, w_M$ 이라고 한다.

$w_i \in V$, 즉 각 단어 $w_i$는 단어장 $V$ 안에 속해있다.

$T$를 품사 모음으로 표시한다. 그리고 $y_i$를 단어 $w_i$의 품사로 표시한다.

이제 단어 $w_i$의 품사에 대한 우리의 예측을 $\hat{y}_i$로 표시한다.

이것이 structured prediction 이고, output이 sequence 형태의 $\hat{y}_1, \cdots, \hat{y}_M$ $(\hat{y}_i \in T)$인 것을 의미한다.

이제 예측을 하기 위해서 LSTM 모델에 문장을 던져줘야 한다. 

$i$ 번째 순서의 hidden state를 $h_i$라고 표시하겠다. 그리고 모든 품사 태그에 고유한 숫자를 부여하겠다. 

이는 지난 번 word embedding 예제에서 word_to_ix를 만들었던 것과 똑같은 이치이다. 

그러면 $\hat{y}_i$를 계산하기 위한 규칙은 다음과 같다.

$$
\hat{y}_i = \arg\max_j \left( \log\text{Softmax}(Ah_i + b) \right)_j
$$

affine map의 hidden state에 log softmax를 취하여 품사 태그를 예측하며, 예측된 태그는 벡터 안에서 가장 큰 값을 갖는다.

Affine map $A$의 target space 차원 값이 $|T|$인 점도 알아두자.

실제 코드로 들어가보겠다. Data를 준비하자.

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


이제 모델을 생성해보자.

In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])
tensor([[-0.0462, -4.0106, -3.6096],
        [-4.8205, -0.0286, -3.9045],
        [-3.7876, -4.1355, -0.0394],
        [-0.0185, -4.7874, -4.6013],
        [-5.7881, -0.0186, -4.1778]])


## Exercise: Augmenting the LSTM part-of-speech tagger with character-level features

위의 예제에서, 각 단어는 embedding을 갖게 되고, embedding이 우리의 sequence 모델의 input으로 수행되었다. 

이제 단어의 문자로부터 파생된 word embedding에 대해 논의해보자.

접두사 같은 문자 단위의 정보는 품사에 큰 영향을 미치기 때문에 이 방법이 크게 도움이 될 것이다.

예를 들어 -ly인 단어는 거의 항상 영어에서 부사로 표시됩니다.

$c_w$를 단어 $w$의 문자 단위의 표현이라고 하겠다. $x_w$는 기존 word embedding을 의미한다.

그러면 sequence 모델의 input으로는 $x_w$와 $c_w$를 합친 것을 넣어줘야 한다.

따라서 $x_w$이 5차원이고 $c_w$가 3차원이라면, LSTM은 8차원의 input을 받아야 할 것이다.

문자 단위의 표현을 얻기 위해서는 LSTM이 단어의 문자들을 받아서 작동시키고, 그 결과인 최종 hidden state를 $c_w$로 하면 된다. 힌트:

새로운 모델은 두 개의 LSTM를 갖게 될 것이다.

POS tag 점수를 산출하는 LSTM과, 각 단어의 문자 단위의 표현을 얻기 위한 LSTM 하나가 더 추가된다.

문자들을 sequence 모델로 돌리기 위해서는 character embedding이 필요할 것이다. 

이 character embedding이 문자를 위한 LSTM에 input으로 들어가게 될 것이다.